In [1]:
import numpy as np
from math import sqrt

## Exercitiul 1

In [17]:
A = np.array([[1., 1., 5.],
               [2., 17., 20.],
               [2., 7., 13.]])
b = np.array([[4.],
               [33.],
               [16.]])
# A = np.array([[1., 0., 0., 0.],
#               [1., 0.524, 0.274, 0.144],
#               [1., 1.047, 1.097, 1.148],
#               [1., 1.57, 2.467, 3.876]])
# b = np.array([[0],
#               [0.866],
#               [0.5],
#               [0]])

In [18]:
def gauss_pt(A, b, eps=1e-13):
    delta = np.linalg.det(A)
    if abs(delta) < eps:
        print("Sistemul nu are solutie unica")
        return None, None, None
    
    n = A.shape[0]
    P = np.identity(n)
    U = A.copy()
    c = b.copy()
    ordine = np.array(range(n))
    for k in range(n-1):
        print()
        print(U)
        print(c)
        # aflam maximul din matricea din dreapta jos. functia ne returneaza numarul elementului 
        p = np.argmax(abs(U[k:,k:]))
        pi = p // (n-k) + k  # calculam linia si coloana. (n-k) e nr de elemente de pe o linie din matricea selectata
        pj = p % (n-k) + k
        
        # schimbam liniile intre ele
        P[pi], P[k] = P[k].copy(), P[pi].copy()
        U[pi], U[k] = U[k].copy(), U[pi].copy()
        c[pi], c[k] = c[k].copy(), c[pi].copy()
        
        # schimbam coloanele pentru U si memoram schimbarea in `ordine`
        U[:,pj], U[:,k] = U[:,k].copy(), U[:,pj].copy()
        ordine[pj], ordine[k] = ordine[k], ordine[pj]
        
        for l in range(k+1, n):
            c[l] = c[l] - (U[l,k] / U[k,k]) * c[k]
            U[l] = U[l] - (U[l,k] / U[k,k]) * U[k]
            
    return P, U, c, ordine

def get_x_gauss_pt(U, b, ordine):
    n = U.shape[0]
    y = np.zeros((n,1))
    x = np.zeros((n,1))
    for i in range(n - 1, -1, -1):
        x[i,0] = (b[i] - U[i,i+1:]@x[i+1:]) / U[i,i]
        
    # aplicam permutarea pe x pentru ca solutiile sunt in alta ordine
    x_ = np.zeros((n,1))
    for i in range(n):
        x_[ordine[i],0] = x[i,0]
    
    return x_

In [19]:
P, U, c, ordine = gauss_pt(A, b)
print(P)
print(U)
print(c)
print(ordine)
if U is not None:
    x = get_x_gauss_pt(U, c, ordine)
    print(x)


[[ 1.  1.  5.]
 [ 2. 17. 20.]
 [ 2.  7. 13.]]
[[ 4.]
 [33.]
 [16.]]

[[20.   17.    2.  ]
 [ 0.   -3.25  0.5 ]
 [ 0.   -4.05  0.7 ]]
[[33.  ]
 [-4.25]
 [-5.45]]
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[20.        17.         2.       ]
 [ 0.        -4.05       0.7      ]
 [ 0.         0.        -0.0617284]]
[[33.        ]
 [-5.45      ]
 [ 0.12345679]]
[2 1 0]
[[-2.]
 [ 1.]
 [ 1.]]


In [548]:
print(A1@x)  # verificare
print((A1@x - b1) < 1e-13)

[[ 47.]
 [ 53.]
 [-89.]
 [-81.]]
[[ True]
 [ True]
 [ True]
 [ True]]


## Exercitiul 2

In [2]:
A2 = np.array([[0., -1., 6., 9.],
               [-7., -10., 9., 6.],
               [-7., -3., -4., -6.],
               [6., -4., -7., 8.]])

In [3]:
def inversa(A, eps=1e-13):
    delta = np.linalg.det(A)
    if abs(delta) < eps:
        print("Determinantul este 0")
        return None
    
    n = A.shape[0]
    U = np.concatenate((A, np.identity(n)), axis=1)  # concatenam I la dreapta
    for k in range(n):
        p = np.argmax(abs(U[k:,k])) + k  # folosim pivotarea pentru a ne asigura ca nu impartim la 0 sau numere f mici
        U[p], U[k] = U[k].copy(), U[p].copy()  # (puteam sa fac asta doar cand erau numere f mici)
        
        if U[k,k] == 0:  # daca peste tot este 0, nu avem la ce imparti
            continue     # verific egalitatea cu == pentru ca valoarea e din matricea initiala
        
        U[k] /= U[k][k]  # facem elementele de pe diagonala principala egale cu 1
        for l in range(n):
            if l == k:   # modificam deasupra si dedesubtul liniei k astfel incat sa ramana 0-uri
                continue # (aplicam acelasi algoritm ca la Gauss, doar ca si pe partea superioara)
            U[l] = U[l] - U[l,k] * U[k]
    
    return U[:,n:]  # returnam doar jumatatea dreapta. cea din stanga e acum matricea identitate
    

In [4]:
print(inversa(A2))

[[-0.18860819  0.06386317 -0.18452848  0.02589048]
 [ 0.178252   -0.1219206   0.07955437 -0.04942727]
 [-0.06464773  0.06182332 -0.10983838 -0.05601757]
 [ 0.17401538 -0.05476228  0.08206496  0.03185313]]


In [5]:
print(A2@inversa(A2))  # verificare. se observa mici erori de calcul

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.33066907e-16  1.00000000e+00  1.11022302e-16  1.11022302e-16]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -1.38777878e-17]
 [ 1.11022302e-16 -1.45716772e-16 -1.11022302e-16  1.00000000e+00]]


In [6]:
aux = A2@inversa(A2)  # transformam valorile f mici in 0 pentru o vizualizare mai buna
aux[aux < 1e-13] = 0
print(aux)  # care e egal cu I

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Exercitiul 3

Folosim factorizarea LU facuta la laborator.

In [3]:
A3 = np.array([[2., 3., 1.],
               [4., 8., 5.],
               [-4., 0., 10.]])
b3 = np.array([[5.],
               [8.],
               [20.]])

In [8]:
def lu_piv(A, eps=1e-13):
    delta = np.linalg.det(A)
    if abs(delta) < eps:
        print("Sistemul nu are solutie unica")
        return None, None, None
    
    n = A.shape[0]
    P = np.identity(n)
    U = A.copy()
    L = np.zeros((n,n))
    for k in range(n - 1):
        p = np.argmax(U[k:,k]) + k
        
        U[p], U[k] = U[k].copy(), U[p].copy()
        P[p], P[k] = P[k].copy(), P[p].copy()
        L[p], L[k] = L[k].copy(), L[p].copy()
        
        L[k+1:,k] = U[k+1:, k] / U[k, k]
        U[k+1:, k:] -= np.outer(L[k+1:, k], U[k,k:])
        
    L += np.identity(n)
    
    # verificare
    if ((P@A - L@U) >= eps).any():
        print("Eroare la calcul")
    
    return P, L, U

def get_x_lu_piv(P, L, U, b):
    n = U.shape[0]
    y = np.zeros((n,1))
    x = np.zeros((n,1))
    Pb = P@b
    # P @ A @ x = P @ b <=>
    # L @ U @ x = P @ b
    # y = U @ x =>
    # y = subs asc (L, Pb)
    for i in range(0, n):
        y[i,0] = (Pb[i] - L[i,:i]@y[:i]) / L[i,i]
        print('---')
        print(y)
    print(y)
    # x = subs desc (U, y)
    for i in range(n - 1, -1, -1):
        x[i,0] = (y[i] - U[i,i+1:]@x[i+1:]) / U[i,i]
    return x

In [9]:
P, L, U = lu_piv(A3)
print(P)
print(L)
print(U)
if U is not None:
    x = get_x_lu_piv(P, L , U, b3)
print(x)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
[[ 1.     0.     0.   ]
 [-1.     1.     0.   ]
 [ 0.5   -0.125  1.   ]]
[[ 4.     8.     5.   ]
 [ 0.     8.    15.   ]
 [ 0.     0.     0.375]]
---
[[8.]
 [0.]
 [0.]]
---
[[ 8.]
 [28.]
 [ 0.]]
---
[[ 8. ]
 [28. ]
 [ 4.5]]
[[ 8. ]
 [28. ]
 [ 4.5]]
[[ 25.]
 [-19.]
 [ 12.]]


In [557]:
print(A3@x)  # verificare
print((A3@x - b3) < 1e-13)

[[  3.]
 [ 15.]
 [-12.]
 [ 69.]]
[[ True]
 [ True]
 [ True]
 [ True]]


## Exercitiul 4

In [560]:
A4 = np.array([[49., 21., -63., 21.],
               [21., 13., -11., 7.],
               [-63., -11., 209., 45.],
               [21., 7., 45., 126.]])

In [561]:
def cholesky(A, eps=1e-15):
    n = A.shape[0]
    
    # conditiile sunt ca A sa fie simetrica si pozitiv definita
    if (A != A.T).any():
        print("Conditiile nu sunt indeplinite")
        return None
    # aplicam criteriul lui Sylvester
    for k in range(n):
        delta = np.linalg.det(A[:k,:k])
        if abs(delta) < eps:
            print("Conditiile nu sunt indeplinite")
            return None
    
    # algoritmul din curs (metoda I)
    L = np.zeros((n,n))
    for k in range(n):
        L[k,k] = sqrt(A[k,k] - sum(L[k,:k] ** 2))
        for i in range(k+1, n):
            L[i,k] = (A[i,k] - L[i,:k]@L[k,:k]) / L[k,k]
            
    return L

In [562]:
L = cholesky(A4)
print(L)

[[ 7.  0.  0.  0.]
 [ 3.  2.  0.  0.]
 [-9.  8.  8.  0.]
 [ 3. -1. 10.  4.]]


In [563]:
print(L@L.T) # verificare
print((L@L.T - A4) < 1e-13)

[[ 49.  21. -63.  21.]
 [ 21.  13. -11.   7.]
 [-63. -11. 209.  45.]
 [ 21.   7.  45. 126.]]
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
